In [1]:
import pickle,random
with open("ngsim_samples_list/ngsim_samples", "rb") as fp:  # Unpickling
        samples_list = pickle.load(fp)
lc_samples = [i for i in samples_list if i[3] == 1]
no_lc_samples = [i for i in samples_list if i[3] == 0]

print(f"The number of lane change scenario: {len(lc_samples)}")
print(f"The number of no lane change scenario: {len(no_lc_samples)}")
print(f"The ratio of lc to no_lc: {len(lc_samples)/len(no_lc_samples)}")


The number of lane change scenario: 320046
The number of no lane change scenario: 7299618
The ratio of lc to no_lc: 0.043844212121785006


In [8]:
from sklearn.model_selection import train_test_split
from ngsim_dataset import NgsimDataset
from torch_geometric.loader import DataLoader
import torch
import pickle,random

lc_samples_file = "ngsim_samples_list/ngsim_lc_samples"
no_lc_samples_file = "ngsim_samples_list/ngsim_no_lc_samples"

with open(lc_samples_file, "rb") as fp:  # Unpickling
    lc_samples_list = pickle.load(fp)

with open(no_lc_samples_file, "rb") as fp:  # Unpickling
    no_lc_samples_list = pickle.load(fp)

lc_samples_train, lc_samples_val = train_test_split(lc_samples_list, test_size=0.1, random_state=42)
no_lc_samples_train, no_lc_samples_val = train_test_split(no_lc_samples_list, test_size=0.1, random_state=42)

samples_train = lc_samples_train + no_lc_samples_train
samples_val = lc_samples_val + no_lc_samples_val

with open(f"ngsim_samples_list/ngsim_samples_train", "wb") as fp:  #Pickling
    pickle.dump(samples_train, fp)
    
with open(f"ngsim_samples_list/ngsim_samples_val", "wb") as fp:  #Pickling
    pickle.dump(samples_val, fp)

# train_set = NgsimDataset(samples_train)
# val_set = NgsimDataset(samples_val)

# torch.set_num_threads(4)
# trainDataloader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
# valDataloader = DataLoader(val_set, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)

In [2]:
rd_samples_train = random.sample(samples_train, 20)
for lc_sample in rd_samples_train:
    print(lc_sample[3])

0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0


In [7]:
import torch
weight = torch.arange(10, 0, -1) / 55
weight = weight.unsqueeze(0).cuda()
print(weight.shape)

lc_mask = np.zeros(16)
lc_mask[3] = 1
lc_mask[8] = 1
lc_weight = [1 if i==0 else 1/lc_ratio for i in lc_mask]
lc_weight = torch.tensor(lc_weight).unsqueeze(1).cuda()
print(lc_weight.shape)


torch.Size([1, 10])
torch.Size([16, 1])


In [3]:
import numpy as np
y_pred = np.zeros((16,10,2))
print(y_pred.shape)
y_gt = y_pred + 1
mask = np.ones_like(y_gt)
lc_mask = np.zeros(16)
lc_mask[3] = 1
lc_mask[8] = 1

lc_ratio = 0.0438
acc = np.zeros_like(mask)
muX = y_pred[:, :, 0]
muY = y_pred[:, :, 1]
x = y_gt[:, :, 0]
y = y_gt[:, :, 1]
out = np.power(x - muX, 2) + np.power(y - muY, 2)
print(out.shape)
# print(out.shape)
# weight = np.arange(10, 0, -1) / 55
# weight = weight.unsqueeze(0).cuda()
# out *= weight

# lc_weight = np.ones_like(lc_mask)
lc_weight = [1 if i==0 else 1/lc_ratio for i in lc_mask]
lc_weight = np.expand_dims(np.array(lc_weight),axis=1)
# print(lc_mask)
print(lc_weight)
out *= lc_weight
print(out)
acc[:, :, 0] = out
acc[:, :, 1] = out
acc = acc * mask
# print(acc)
lossVal = np.sum(acc) / np.sum(mask)
print(lossVal)


(16, 10, 2)
(16, 10)
[[ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [22.83105023]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [22.83105023]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]]
[[ 2.          2.          2.          2.          2.          2.
   2.          2.          2.          2.        ]
 [ 2.          2.          2.          2.          2.          2.
   2.          2.          2.          2.        ]
 [ 2.          2.          2.          2.          2.          2.
   2.          2.          2.          2.        ]
 [45.66210046 45.66210046 45.66210046 45.66210046 45.66210046 45.66210046
  45.66210046 45.66210046 45.66210046 45.66210046]
 [ 2.          2.          2.          2.          2.          2.
   2.          2.          2.          2.        ]
 [ 2.          2.          2.          2.          2.          2.
   2.          2.          2.          2.        ]
 [ 2.          2.          2

In [6]:
with open(f"ngsim_samples_list/ngsim_lc_samples", "wb") as fp:  #Pickling
    pickle.dump(lc_samples, fp)
    
with open(f"ngsim_samples_list/ngsim_no_lc_samples", "wb") as fp:  #Pickling
    pickle.dump(no_lc_samples, fp)

In [2]:
rd_lc_samples = random.sample(lc_samples, 10)
for lc_sample in rd_lc_samples:
    print(lc_sample)

[5, 2049, 911, 1, 927, 902, 926, 914, 907]
[6, 7685, 1932, 1, 1933, 1924, 2002, 1972, 1915, 1956, 1941, 1934]
[5, 3899, 1338, 1, 1368, 1358, 1345, 1333, 1329, 1373, 1367, 1352]
[2, 9266, 2512, 1, 2517, 2507, 2499]
[6, 2682, 491, 1, 3008, 3006, 534, 493, 484, 506, 494, 487]
[5, 7312, 2513, 1, 2538, 2583, 2517, 2389, 2503, 2670, 2641, 2642]
[5, 1111, 673, 1, 662, 650, 647, 701, 693]
[4, 6121, 2126, 1, 2131, 2119, 2129, 2108, 2102, 2116, 2107, 2104]
[3, 454, 4, 1, 8, 13, 5, 1]
[4, 5905, 1933, 1, 1939, 1925, 1930, 1924, 1916, 1935, 1929, 1921]


In [5]:
import pandas as pd
df = pd.read_csv(f"data_i80/trajectories-0500pm-0515pm.csv")
df = df[(df['Vehicle_ID']==2513) & (df['Frame_ID']>=7282) & (df['Frame_ID']<=7362)]
print(df)

         Vehicle_ID  Frame_ID  Total_Frames    Global_Time  Local_X  Local_Y  \
1310788        2513      7282          1563  1113437495100   26.152  145.427   
1310789        2513      7283          1563  1113437495200   26.078  146.632   
1310790        2513      7284          1563  1113437495300   26.003  147.837   
1310791        2513      7285          1563  1113437495400   25.930  149.041   
1310792        2513      7286          1563  1113437495500   25.856  150.241   
...             ...       ...           ...            ...      ...      ...   
1310864        2513      7358          1563  1113437502700   30.944  204.172   
1310865        2513      7359          1563  1113437502800   30.978  205.207   
1310866        2513      7360          1563  1113437502900   31.012  206.242   
1310867        2513      7361          1563  1113437503000   31.045  207.277   
1310868        2513      7362          1563  1113437503100   31.079  208.310   

            Global_X     Global_Y  v_Le

In [9]:
ngsim_samples_file = "ngsim_samples_list/ngsim_samples"
with open(ngsim_samples_file, "rb") as fp:  # Unpickling
    ngsim_samples_list = pickle.load(fp)

print(len(ngsim_samples_list))

7619664


In [11]:
print(ngsim_samples_list[:100:5])

[[1, 44, 2, 0], [1, 49, 2, 0], [1, 54, 2, 0, 13], [1, 59, 2, 0, 13], [1, 64, 2, 0, 13], [1, 69, 2, 0, 13, 10], [1, 74, 2, 0, 13, 10], [1, 79, 2, 0, 13, 10, 9], [1, 84, 2, 0, 13, 12, 10, 9], [1, 89, 2, 0, 13, 12, 10, 9], [1, 94, 2, 0, 13, 12, 10, 9], [1, 99, 2, 0, 13, 12, 10, 9], [1, 104, 2, 0, 13, 12, 10, 20, 9], [1, 109, 2, 0, 13, 12, 10, 20, 9], [1, 114, 2, 0, 13, 12, 10, 20, 9], [1, 119, 2, 0, 13, 12, 10, 20, 9], [1, 124, 2, 0, 13, 12, 10, 20, 9], [1, 129, 2, 0, 13, 12, 10, 20, 9], [1, 134, 2, 0, 13, 12, 10, 20, 9], [1, 139, 2, 0, 13, 12, 10, 20, 9]]


In [2]:
import pandas as pd
import numpy as np
from plot_helper import find_files
csvs = find_files(f"/home/jiang/trajectory_pred/GNN-RNN-Based-Trajectory-Prediction-ITSC2021/data_i80",suffix='csv')
csvs += find_files(f"/home/jiang/trajectory_pred/GNN-RNN-Based-Trajectory-Prediction-ITSC2021/data_us101",suffix='csv')
print(csvs)

['/home/jiang/trajectory_pred/GNN-RNN-Based-Trajectory-Prediction-ITSC2021/data_i80/trajectories-0400pm-0415pm.csv', '/home/jiang/trajectory_pred/GNN-RNN-Based-Trajectory-Prediction-ITSC2021/data_i80/trajectories-0500pm-0515pm.csv', '/home/jiang/trajectory_pred/GNN-RNN-Based-Trajectory-Prediction-ITSC2021/data_i80/trajectories-0515pm-0530pm.csv', '/home/jiang/trajectory_pred/GNN-RNN-Based-Trajectory-Prediction-ITSC2021/data_us101/trajectories-0750am-0805am.csv', '/home/jiang/trajectory_pred/GNN-RNN-Based-Trajectory-Prediction-ITSC2021/data_us101/trajectories-0805am-0820am.csv', '/home/jiang/trajectory_pred/GNN-RNN-Based-Trajectory-Prediction-ITSC2021/data_us101/trajectories-0820am-0835am.csv']


In [5]:
dfs = [pd.read_csv(csv) for csv in csvs]

# mean_vel_x = 
total_cnt = 0
vel_x = 0
vel_y = 0
for df in dfs:
    total_cnt += len(df)
    vel_x += np.abs(df['Local_X'].values[1:] - df['Local_X'].values[0:-1]).sum()
    vel_y += np.abs(df['Local_Y'].values[1:] - df['Local_Y'].values[0:-1]).sum()
    
mean_vel_x = vel_x / total_cnt
mean_vel_y = vel_y / total_cnt

print(mean_vel_x, mean_vel_y,mean_vel_y/mean_vel_x)
    
    

0.07532491751366617 5.232584958593724 69.4668528199102
